In [ ]:
!pip install -q gdown
import gdown
import re
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Install necessary libraries
!pip install -q google-api-python-client google-auth-httplib2 google-auth-oauthlib

import io
import re
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

In [ ]:
!pip install -q gdown
!pip install -q google-api-python-client google-auth-httplib2 google-auth-oauthlib
import os
import gdown
from google.oauth2 import service_account
from googleapiclient.discovery import build


# Authenticate and create the Drive API client
auth.authenticate_user()
drive_service = build('drive', 'v3')


In [ ]:
import re

# Function to extract folder ID from the Google Drive folder URL
def extract_folder_id(drive_url):
    match = re.search(r'/folders/([a-zA-Z0-9_-]+)', drive_url)
    return match.group(1) if match else None

# Replace with your actual Google Drive folder URL
drive_link = 'https://drive.google.com/drive/folders/1a53SSsUor0_cVv-_vRo0zp5kBCiU2rqQ?usp=sharing'

# Extract folder ID from the link
folder_id = extract_folder_id(drive_link)
print(f"Folder ID: {folder_id}")



Folder ID: 1a53SSsUor0_cVv-_vRo0zp5kBCiU2rqQ


In [ ]:
# Function to list image files in the folder
def list_images_in_folder(folder_id):
    query = f"'{folder_id}' in parents"
    results = drive_service.files().list(q=query).execute()
    items = results.get('files', [])
    return [item['id'] for item in items if item['mimeType'].startswith('image/')]  # Filter only images

# Get list of image file IDs
image_ids = list_images_in_folder(folder_id)
print(f"Found {len(image_ids)} images.")


Found 0 images.


In [ ]:
import io
import cv2
import numpy as np
from googleapiclient.http import MediaIoBaseDownload
from PIL import Image

# Function to download the image from Google Drive into memory
def download_image_in_memory(image_id):
    request = drive_service.files().get_media(fileId=image_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
    fh.seek(0)
    return Image.open(fh)  # Return as PIL image for easier manipulation


In [ ]:
# Function to segment cracks in images using OpenCV
def segment_cracks(img, min_crack_length=20):
    equalized = cv2.equalizeHist(img)
    blurred = cv2.GaussianBlur(equalized, (5, 5), 0)
    edges = cv2.Canny(blurred, 30, 150)
    kernel = np.ones((2, 2), np.uint8)
    morphed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel, iterations=1)

    contours, _ = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    mask = np.zeros_like(img)
    crack_id = 1
    for cnt in contours:
        if cv2.arcLength(cnt, True) > min_crack_length:
            cv2.drawContours(mask, [cnt], -1, crack_id, -1)
            crack_id += 1

    return mask, morphed, contours

# Process each image from the Google Drive folder (in-memory)
for image_id in image_ids:
    pil_img = download_image_in_memory(image_id)  # Download the image as a PIL object
    img = np.array(pil_img.convert("L"))  # Convert PIL image to grayscale OpenCV format

    # Run segmentation on the image
    seg_mask, edge_img, contours = segment_cracks(img)

    # Create a color overlay for visualization
    color_overlay = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    unique_cracks = np.unique(seg_mask)
    colors = [tuple(np.random.randint(0, 255, 3).tolist()) for _ in unique_cracks]

    for cid in unique_cracks:
        if cid == 0: continue
        color_overlay[seg_mask == cid] = colors[cid % len(colors)]

    # Plot results
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 3, 1), plt.imshow(img, cmap='gray'), plt.title('Original X-ray')
    plt.subplot(1, 3, 2), plt.imshow(edge_img, cmap='gray'), plt.title('Edge Detection')
    plt.subplot(1, 3, 3), plt.imshow(color_overlay), plt.title(f'Segmented Cracks ({len(unique_cracks)-1} found)')
    plt.tight_layout()
    plt.show()